<a href="https://colab.research.google.com/github/accarter/DS-Unit-1-Sprint-4-Build-1/blob/master/Unit_1_Build_Video_Game_Sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DS Unit 1 Sprint 4 Build 1 - Video Game Sales

## Data Exploration and Cleaning

In [1]:
import pandas as pd

In [2]:
url = 'https://raw.githubusercontent.com/accarter/portfolio/master/assets/vgsales.csv'
vg = pd.read_csv(url)
vg.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [3]:
# remove Global_Sales column
vg = vg.drop('Global_Sales', axis=1)

# restructure DataFrame 
vg_tidy = pd.melt(vg, id_vars= vg.columns[:6], var_name="Region", value_name="Sales")

# rename regions
vg_tidy['Region'] = vg_tidy['Region'].replace({
    'NA_Sales': 'North America',
    'EU_Sales': 'Europe',
    'JP_Sales': 'Japan',
    'Other_Sales': 'Other'
})

vg_tidy.head()

,Rank,Name,Platform,Year,Genre,Publisher,Region,Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,North America,41.49
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,North America,29.08
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,North America,15.85
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,North America,15.75
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,North America,11.27


In [7]:
vg_tidy.shape

(66392, 8)

In [8]:
vg_tidy.isnull().sum()

Rank            0
Name            0
Platform        0
Year         1084
Genre           0
Publisher     232
Region          0
Sales           0
dtype: int64

In [9]:
vg_tidy.describe()

,Rank,Year,Sales
count,66392.000000,65308.000000,66392.000000
mean,8300.605254,2006.406443,0.134291
std,4791.745667,5.828847,0.519930
min,1.000000,1980.000000,0.000000
25%,4151.000000,2003.000000,0.000000
50%,8300.500000,2007.000000,0.010000
75%,12450.000000,2010.000000,0.090000
max,16600.000000,2020.000000,41.490000


In [10]:
vg_tidy.describe(exclude='number')

,Name,Platform,Genre,Publisher,Region
count,66392,66392,66392,66160,66392
unique,11493,31,12,578,4
top,Need for Speed: Most Wanted,DS,Action,Electronic Arts,Japan
freq,48,8652,13264,5404,16598


## Data Visualization

In [4]:
import plotly.graph_objects as go

genres = list(vg_tidy['Genre'].unique())

data = []
for region in vg_tidy['Region'].unique():
    y = [vg_tidy[(vg_tidy['Genre'] == genre) & (vg_tidy['Region'] == region)]['Sales'].sum() for genre in genres]
    data.append(go.Bar(name=region, x=genres, y=y, text=y))

fig = go.Figure(data=data)
fig.update_layout(barmode='stack')
fig.show()

In [5]:
vg_global = pd.DataFrame(vg_tidy.groupby(['Name', 'Year', 'Genre'])['Sales'].sum()).reset_index()

data = []

for genre in vg_tidy['Genre'].unique():
    vg_global_by_genre = vg_global[vg_global['Genre'] == genre]
    data.append(go.Scatter(x=vg_global_by_genre['Year'],
                           y=vg_global_by_genre['Sales'],
                           mode='markers',
                           name=genre,
                           text=vg_global_by_genre['Name']))
  
fig = go.Figure(data=data)
fig.show()

In [11]:
vg_00 = vg_tidy[(vg_tidy['Year'] >= 2000) & (vg_tidy['Year'] < 2010)]
G = vg_00.groupby(['Region', 'Genre'])['Sales']
vg_00_glob = G.sum().unstack(level=['Genre']).reset_index()
vg_00_glob.head()

Genre,Region,Action,Adventure,Fighting,Misc,Platform,Puzzle,Racing,Role-Playing,Shooter,Simulation,Sports,Strategy
0,Europe,235.80,31.00,54.64,131.94,102.59,33.72,138.33,81.27,123.51,79.14,223.94,20.30
1,Japan,60.02,21.67,31.00,61.14,39.46,20.78,21.07,138.17,9.24,26.86,60.37,20.91
2,North America,463.12,62.04,122.06,242.93,203.63,55.64,226.76,171.19,257.93,136.86,433.84,32.91
3,Other,99.37,9.26,23.56,50.96,31.26,8.67,54.30,27.60,43.71,23.16,87.16,5.72


In [12]:
import plotly.graph_objects as go
import numpy as np

data = []
for genre in vg_tidy['Genre'].unique():
    data.append(go.Bar( x=vg_00_glob['Region'], y=vg_00_glob[genre], name=genre))
    
fig = go.Figure(data=data)

fig.show()

## Inferential Statistics

### Comparison of average sales figures between all possible combinations of genres using two-sample independent t-tests

In [13]:
import scipy.stats as stats

def ttest_genres(genre1, genre2, alpha):
    def sales_for_genre(genre):
        return vg_tidy[vg_tidy['Genre'] == genre]['Sales']
    
    
    if genre1 == genre2: return ''
    
    pval = stats.ttest_ind(sales_for_genre(genre1),
                           sales_for_genre(genre2), 
                           nan_policy='omit')[1]
    
    if pval < alpha: return ''
    return '{:.2g}'.format(pval)

genres = list(vg_tidy['Genre'].unique())
ttest_dict = {genre1: [ttest_genres(genre1, genre2, 0.05) for genre2 in genres] for genre1 in genres}
ttest_dict['Genres']  = genres
pd.DataFrame(ttest_dict).set_index('Genres')

,Sports,Platform,Racing,Role-Playing,Puzzle,Misc,Shooter,Simulation,Action,Fighting,Adventure,Strategy
Genres,,,,,,,,,,,,
Sports,,,0.67,0.18,,,,,0.17,0.44,,
Platform,,,,,,,,,,,,
Racing,0.67,,,0.36,,,,,,0.16,,
Role-Playing,0.18,,0.36,,,,,,,,,
Puzzle,,,,,,0.33,,0.53,,,,
Misc,,,,,0.33,,,0.71,,0.061,,
Shooter,,,,,,,,,,,,
Simulation,,,,,0.53,0.71,,,,,,
Action,0.17,,,,,,,,,0.96,,


### Chi-square tests to determine relationship between tastes between all possible cominbations of regions

In [14]:
def chi2_region(df, region1, region2):
    table = pd.concat([
        df[df['Region'] == region1],
        df[df['Region'] == region2]
    ]).set_index('Region')
    
    chi2, pval, dof, expected = stats.chi2_contingency(table.iloc[:,1:])
    return '{:.2g}'.format(pval)

regions = list(vg_tidy['Region'].unique())
chi2_dict = {region1: [chi2_region(vg_00_glob, region1, region2) 
                       for region2 in regions] 
             for region1 in regions}
chi2_df = pd.DataFrame(chi2_dict)
chi2_df['Regions']  = regions
chi2_df = pd.DataFrame(chi2_df).set_index('Regions')
chi2_df

,North America,Europe,Japan,Other
Regions,,,,
North America,1,0.84,6.4e-44,0.78
Europe,0.84,1,5e-36,0.94
Japan,6.4e-44,5e-36,1,4.7e-23
Other,0.78,0.94,4.7e-23,1
